# Intento 4

| Score | Estimación _holdout_ | Timestamp |
| --- | --- | --- |
| **0.8592** | 0.8647 | 2022-01-01 22:16:21 UTC |

- Estrategia de preprocesado básica (del profesor).

In [5]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb

# código común
import common.preprocessing as pre
from common.vars import SEED

# display all columns
pd.options.display.max_columns = None

In [6]:
train_features = pd.read_csv('../data/flu_training_set_features.csv', index_col='respondent_id')
train_labels = pd.read_csv('../data/flu_training_set_labels.csv', index_col='respondent_id')
test_features = pd.read_csv('../data/flu_test_set_features.csv', index_col='respondent_id')

In [7]:
def preprocess(train_features, test_features):
  """PREPROCESAMIENTO
  """
  labels = {}
  imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
  _train_features = train_features.astype(str)
  # uso SimpleImputer para "eliminar los nulos", como elimina los nombres de las columnas los ponemos de nuevo
  _train_features = pd.DataFrame(imp.fit_transform(_train_features), index=train_features.index)
  _train_features.columns = train_features.columns

  _all = pd.concat([train_features, test_features])

  # aprendo las etiquetas
  for col in _all.columns:
    labels[col] = LabelEncoder()
    labels[col].fit(_all[col].astype(str))
  
  # aplico el etiquetado
  for col in _train_features.columns:
    _train_features[col] = labels[col].transform(_train_features[col])
  
  _test_features = test_features.astype(str)
  _test_features = pd.DataFrame(imp.transform(_test_features), index=test_features.index)
  _test_features.columns = test_features.columns

  # aplico el mismo etiquetado con los valores de test
  for col in _test_features.columns:
    _test_features[col] = labels[col].transform(_test_features[col])
  
  return _train_features, _test_features

In [8]:
train_features, test_features = preprocess(train_features, test_features)

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

def cross_validation(model, X, y, cv):
  measures = []
  time_total_0 = time.time()

  for train, test in cv.split(X, y):
    X_train = X.iloc[train, :].values
    X_test = X.iloc[test, :].values
    y_train = y.iloc[train, :].values
    y_test = y.iloc[test, :].values
    time_iter_0 = time.time()
    _model = model.fit(X_train, y_train)
    time_iter = time.time() - time_iter_0
    preds = _model.predict_proba(X_test)
    y_preds = pd.DataFrame(
      {
        'h1n1_vaccine':     preds[0][:,1],
        'seasonal_vaccine': preds[1][:,1],
      },
      index = y.iloc[test,:].index
    )
    measure = roc_auc_score(y_test, y_preds)
    print(f"[CV] AUC score: {measure:.4f}. Time: {time_iter:6.2f}")
    measures.append(measure)
  
  time_total = time.time() - time_total_0
  print(f"[CV] Average AUC: {np.mean(measures):.4f}. Total time: {time_total:6.2f}")
  return model, y_preds

def preds_to_df(preds, features):
  return pd.DataFrame(
    {
      'h1n1_vaccine': preds[0][:,1],
      'seasonal_vaccine': preds[1][:,1]
    },
    index = features.index
  )

### _Holdout_ como proxy de ROC en test

In [10]:
# split train features in two
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=SEED)

In [11]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', n_estimators=200)
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, X_train, y_train, kf)

[CV] AUC score: 0.8636. Time:   0.58
[CV] AUC score: 0.8641. Time:   0.48
[CV] AUC score: 0.8570. Time:   0.56
[CV] AUC score: 0.8544. Time:   0.77
[CV] AUC score: 0.8698. Time:   0.87
[CV] Average AUC: 0.8618. Total time:   3.56


In [12]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(X_train, y_train)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(X_test)

In [13]:
roc_auc_score(y_test, preds_to_df(lgbm_preds, X_test))

0.8646884663508434

### Submission

In [14]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', n_estimators=200)
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, train_features, train_labels, kf)

[CV] AUC score: 0.8647. Time:   0.60
[CV] AUC score: 0.8653. Time:   0.77
[CV] AUC score: 0.8624. Time:   1.11
[CV] AUC score: 0.8650. Time:   1.11
[CV] AUC score: 0.8603. Time:   1.66
[CV] Average AUC: 0.8635. Total time:   5.69


In [15]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(train_features, train_labels)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(test_features)

In [17]:
preds_to_df(lgbm_preds, test_features).to_csv('../submissions/04.csv')